Iteradores e geradores
===

Temos visto uma variedade de tipos em Python que se comportam como sequências, ou seja, onde podemos começar com um elemento e percorrer os vários elementos do dado sucessivamente.  Exemplos de sequência incluem listas, tupla, strings, dicionários e conjuntos.

Esta ideia de sequência pode ser generalizada através dos iteradores, que são objetos que contém o método `__next__()`, que retorna o próximo elemento.  Podemos pensar nos iteradores como ponteiros para acesso sequencial aos elementos de um container, como por exemplo, uma lista.  O método `__next__()` pode ser chamado usando a função `next()` no iterador.

O container, da sua parte, define o método `__iter__()` que retorna um iterador para ele.  De forma semelhante a `next()`, podemos chamar o método `__iter__()` usando a função `iter()` no iterador.

Observe que diferentes iteradores de um container são independentes entre si.

In [ ]:
a = [1, 2, 3]
b1 = a.__iter__()
b2 = iter(a)
print("O 1º valor de a através de b1:", next(b1))
print("O 2º valor de a através de b1:", next(b1))
print("O 3º valor de a através de b1:", next(b1))
print("O 1º valor de a através de b2:", next(b2))
print("O 2º valor de a através de b2:", next(b2))
print("O 3º valor de a através de b2:", next(b2))

In [ ]:
# Quando o iterador chega ao final do container ele lança uma
# exceção do tipo StopIteration
try:
    print(next(b1))
except StopIteration:
    print("Final do container")

In [ ]:
# Uma classe definindo um iterador e outra usando-a como iterador
# Lembre-se que o método __next__() deve lançar a exceção
# StopIteration quando tiver chegado ao fim do container
class MyIter:
    
    def __init__(self, container):
        self.container = container
        self.idx = -1
    
    def __next__(self):
        self.idx += 1
        if self.idx >= len(self.container):
            raise StopIteration
        return self.container[self.idx]
    
class MyContainer:
    
    def __init__(self, iv):
        self.container = list(iv)
        
    def __iter__(self):
        return MyIter(self.container)
    

c = MyContainer(set([1, 2, 3, 4, 5]))
p = iter(c)
print("1º valor de c é", next(p))
print("2º valor de c é", next(p))
print("3º valor de c é", next(p))
print("4º valor de c é", next(p))
print("5º valor de c é", next(p))
try:
    print("1º valor de c é", next(p))
except StopIteration:
    print("Final de iteração")
try:
    print("1º valor de c é", next(p))
except StopIteration:
    print("Final de iteração")

In [ ]:
# É muito comum, entretanto, que a classe container seja o seu
# próprio iterador.  Isto é feito fazendo com que a classe
# container contenha ambos os métodos __iter__() e __next__(),
# onde o 1º método retorna `self`
class NumerosPares:
    def __init__(self, max_n):
        self.n = 0
        self.max_n = max_n
        
    def __next__(self):
        if self.n >= self.max_n:
            raise StopIteration
        res = self.n
        self.n += 2
        return res
    
    def __iter__(self):
        return self
    
c = NumerosPares(5)
p = iter(c)
print("c == p?", p is c)
print("1º número par:", next(p))
print("2º número par:", next(p))
print("3º número par:", next(p))
try:
    print("4º número par:", next(p))
except StopIteration:
    print("Fim de iteração")

Existem várias construções que sabem usar containers e seus iteradores.  O principal que vimos até agora é o `for`, tanto na sua versão usual como em list comprehensions.  Além destes, vimos o método `join()` de strings e os construtores de lista, tuplas, conjuntos e dicionários.  Todos estes sabem lidar com a exceção StopIteration lançada pelo iterador.

In [ ]:
# Exemplo clássico de consumo de um iterador
print("Números pares menores do que 21:", end=" ")
for n in NumerosPares(21):
    print(n, end=" ")
print()

# Outro exemplo muito comum
lst = [str(n) for n in NumerosPares(30)]
print("Números pares menores do que 30:", " ".join(lst))

# Outro exemplo
s1 = set(NumerosPares(51))
s2 = set(range(3, 51, 3))  # Múltiplos de 3 menores do que 51
print("Múltiplos pares de 3 menores do que 51:", " ".join(str(n) for n in s1 & s2))

Com iteradores podemos fazer coisas poderosas do tipo ter um container que é potencialmente infinito em extensão e ainda assim não entrar em loop infinito.

In [ ]:
# Estendamos a classe de números pares acima para ser potencialmente
# infinita.  Usaremos a função islice() do módulo itertools para
# selecionar apenas alguns elementos.  Observe que com um iterador
# não podemos usar o slice normal a não ser que convertamos primeiro
# para uma lista, o que se o iterador for infinito não vamos querer
# fazer
# Referência: file:///usr/share/doc/python3/html/library/itertools.html#itertools.islice
class NumerosPares:
    def __init__(self):
        self.n = 0
        
    def __next__(self):
        res = self.n
        self.n += 2
        return res
    
    def __iter__(self):
        return self

from itertools import islice

c = NumerosPares()
print("Números pares de índices entre 1000000 e 1000010:", end=" ")
for n in islice(c, 1000000, 1000010):
    print(n, end=" ")

#### Exercício

Defina um container que retorne os números de Fibonacci.

In [ ]:
# Testes para o gerador fibonacci
import unittest
from itertools import islice

#
# Defina o iterator fibonacci aqui
#

class MyTest(unittest.TestCase):

    def test(self):
        self.assertEqual(list(islice(fibonacci(), 400, 403)), 
                         [176023680645013966468226945392411250770384383304492191886725992896575345044216019675,
                          284812298108489611757988937681460995615380088782304890986477195645969271404032323901,
                          460835978753503578226215883073872246385764472086797082873203188542544616448248343576])

unittest.main(argv=['first-arg-is-ignored'], exit=False)

Geradores
===

Geradores é uma maneira de se criar iteradores que é ao mesmo tempo simples e intuitiva.  Basicamente um gerador tem a forma de uma função mas usa a instrução `yield` ao invés de `return`.  Esta instrução retorna o valor que é passado a ela para quem a chamou mas guarda o estado da função retornando a ele quando se chama `next()` no seu iterador.

Se a função terminar, ela automaticamente lança a exceção StopIteration.

In [ ]:
# Vamos converter a classe NumerosPares para geradores
def NumerosPares():
    n = 0
    while 1:
        yield n
        n += 2

from itertools import islice

print("Números pares de índices entre 1000000 e 1000010:", end=" ")
for n in islice(NumerosPares(), 1000000, 1000010):
    print(n, end=" ")

#### Exercício

Defina um gerador que retorne os números de Fibonacci.

In [ ]:
# Testes para o gerador fibonacci
import unittest
from itertools import islice

class MyTest(unittest.TestCase):

    def test1(self):
        self.assertEqual(list(islice(fibonacci(), 1)), [0])

    def test2(self):
        self.assertEqual(list(islice(fibonacci(), 1, 2)), [1])

    def test3(self):
        self.assertEqual(list(islice(fibonacci(), 20, 23)), [6765, 10946, 17711])

unittest.main(argv=['first-arg-is-ignored'], exit=False)

Expressões de geradores
===

A criação de geradores pode ficar ainda mais simples se usarmos as *expressões de geradores*, que nada mais são do que uma list comprehension envolta em parênteses ao invés de colchetes.

In [ ]:

for i in (n*n for n in range(1, 10)):
    print(i, end=" ")
print()

In [ ]:
# A mesma expressão de gerador acima para criar a string completa antes de imprimi-la
print(" ".join(str(n*n) for n in range(1, 10)))

#### Exercício

Usando expressões de geradores, crie uma função que retorne uma string contendo um tabuleiro tipo tabuleiro de xadrez de dimensão $n$.  Veja o exemplo abaixo para um tabuleiro de tamanho 3 (tipo jogo da velha).

     --- --- --- 
    |   |   |   |
     --- --- --- 
    |   |   |   |
     --- --- --- 
    |   |   |   |
     --- --- ---`

In [ ]:
import unittest
from itertools import islice

class MyTest(unittest.TestCase):

    def test1(self):
        expected = """ --- --- --- 
|   |   |   |
 --- --- --- 
|   |   |   |
 --- --- --- 
|   |   |   |
 --- --- --- """
        self.assertEqual(tabuleiro(3), expected)

    def test1(self):
        expected = """ --- --- --- --- 
|   |   |   |   |
 --- --- --- --- 
|   |   |   |   |
 --- --- --- --- 
|   |   |   |   |
 --- --- --- --- 
|   |   |   |   |
 --- --- --- --- """
        self.assertEqual(tabuleiro(4), expected)

unittest.main(argv=['first-arg-is-ignored'], exit=False)